# ReadDesignChain LLM Eval

In [ ]:
%load_ext autoreload
%autoreload all

## Run the chain

In [ ]:
from meche_copilot.schemas import *
from meche_copilot.chains.read_design_chain import ReadDesignChain
from meche_copilot.utils.config import load_config, find_config
from langchain.callbacks import StdOutCallbackHandler

In [ ]:
config = SessionConfig.from_yaml(find_config('/workspaces/meche-copilot/session-config.yaml'))
sess = Session.from_config(config=config)

In [ ]:
chain = ReadDesignChain()
chain.design_data_cache

In [ ]:
# SHOULD FAIL - NOT IMPLEMENTED
# chain.read_design_drawings(scoped_eq=sess.equipments)

In [ ]:
# show your work so we can eval camelot plots and such
design_schedules = chain.read_design_schedules(scoped_eq=sess.equipments, show_your_work=True)

## Did LLM select the correct schedules from the design ref docs provided?

should find a lot but scope 4

In [ ]:
from meche_copilot.utils.converters import pydantic_from_jsonl
all_scheds = pydantic_from_jsonl(chain.design_data_cache / '1_all_design_schedules.jsonl', EngineeringDesignSchedule)
len(all_scheds)

In [ ]:
scoped_scheds = pydantic_from_jsonl(chain.design_data_cache / '2_scoped_design_schedules_with_titles.jsonl', EngineeringDesignSchedule)
len(scoped_scheds)

In [ ]:
for i, eds in enumerate(scoped_scheds):
    print(i, eds.equipment_name, eds.title)

In [ ]:
import pandas as pd
from meche_copilot.utils.converters import title_to_filename
fname = title_to_filename(scoped_scheds[0].title)
result_decision_matrix = pd.read_csv(chain.design_data_cache / f"{fname}_results.csv", index_col=0)
result_decision_matrix

In [ ]:
df = pd.read_parquet(chain.design_data_cache / f"{fname}.parquet")
df

## Did camelot select the right table rects and get the right data for each schedule?

In [ ]:
from meche_copilot.utils.envars import DATA_CACHE
camelot_plots_fpaths = [f for f in Path(DATA_CACHE / 'camelot_plots').glob('*.png')]
camelot_plots_fpaths

In [ ]:
from meche_copilot.utils.converters import title_to_filename
from helpers import display_images_mpl
class Results:
    def __init__(self, title, df, image_fpaths):
        self.title = title
        self.fname = title_to_filename(title)
        self.df = df
        self.image_fpaths = image_fpaths

    def show_images(self):
        display_images_mpl(self.image_fpaths, width=6, height=4)

res = []
for eds in scoped_scheds:
    fname = title_to_filename(eds.title)
    df_fpath = chain.design_data_cache / f'{fname}.parquet'
    df = None
    if df_fpath.exists():
        df = pd.read_parquet(chain.design_data_cache / f'{fname}.parquet')
    image_fpaths = []
    for cam_fname in camelot_plots_fpaths:
        if str(cam_fname.name).startswith(str(fname)):
            image_fpaths.append(cam_fname)
    res.append(Results(eds.title, df, image_fpaths))

In [ ]:
i = 0
print(res[i].title)
print(res[i].show_images())
res[i].df

## How well does the combiner combine schedule results?